In [1]:
# ! pip install transformers
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer

import torchsummary as summary
from tqdm import tqdm

from torch import nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os
import base64
import gc
import glob, os
import random

from torch.optim import Adam
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import json
from general_functions import *


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Import data
# Access aws credentials from json file
with open("../aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Specify s3 bucket
bucket = "fs-reghub-news-analysis"

# Connect to aws and dowload the files
aws = awsOps(aws_creds_json)
df = aws.get_df(bucket=bucket, file="data_rule_labels_v1.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df[['news_content']]
df = df.reset_index()
del df['index']
df=df[:40]
df

,news_content
0,Berenberg Bank analysts have provided a buy ra...
1,"The article states that Berenberg, a German in..."
2,"In their analysis on October 30, 2023, experts..."
3,"Berenberg Bank has issued a \""buy\"" recommenda..."
4,The private bank Berenberg has upgraded its ra...
5,In a research note published by Sebastian Bray...
6,"Der renommierte Volkswirt Holger Schmieding, d..."
7,In relation to Credit Suisse's emergency takeo...
8,The article features an exclusive interview wi...
9,The article discusses the recommendation made ...


In [3]:
def dynamic_whole_word_masking(df=df, column_name='news_content', mask_probability=0.15):
    # Initialize BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Function to mask tokens of a sentence
    def mask_sentence(sentence):
        tokens = tokenizer.tokenize(sentence)
        new_tokens = []
        mask_indices = random.sample(range(len(tokens)), int(len(tokens) * mask_probability))

        for i, token in enumerate(tokens):
            # Check if the token is part of a word to be masked
            if i in mask_indices or (i > 0 and tokens[i-1] in mask_indices and token.startswith('##')):
                new_tokens.append('[MASK]')
            else:
                new_tokens.append(token)

        return tokenizer.convert_tokens_to_string(new_tokens)

    # Apply masking to each row in the DataFrame
    df[column_name] = df[column_name].apply(mask_sentence)

    return df


In [4]:
# import BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df,transform=dynamic_whole_word_masking):
        self.labels = [tokenizer(text,padding='max_length',truncation=True,return_tensors="pt") for text in df['news_content']]
        self.transform=transform
        if self.transform:
            df1=self.transform(df=df)
        self.text=[tokenizer(text,padding='max_length',truncation=True,return_tensors="pt") for text in df1['news_content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx]

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_texts, batch_labels

In [5]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [6]:
df_train, df_val = train_test_split(df[['news_content']], test_size=0.25)

In [7]:
df_test=0

In [8]:
from transformers import AutoTokenizer, BertForMaskedLM
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model=model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)

df_val=df_val.reset_index()
df_val.drop('index',axis=1,inplace=True)
df_val

,news_content
0,The article highlights the start of the annual...
1,In a research note published by Sebastian Bray...
2,This article is about the holdings and ratings...
3,Berenberg Bank has downgraded Reckitt Benckise...
4,"The article states that Berenberg, a German in..."
5,German bank Berenberg has reduced its price ta...
6,"Deutsche Bank AG has acquired an additional 1,..."
7,Die CommerzbankAktie befindet sich derzeit in ...
8,"According to Commerzbank Analyst Martin Roth, ..."
9,The article discusses the current situation of...


In [10]:
'''
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n
'''

'\n# display BERT layers\nn=0\nfor x in model.state_dict():\n    n=n+1\n    print(x)\nn\n'

In [11]:
df

,news_content
0,Berenberg Bank analysts have provided a buy ra...
1,"The article states that Berenberg, a German in..."
2,"In their analysis on October 30, 2023, experts..."
3,"Berenberg Bank has issued a \""buy\"" recommenda..."
4,The private bank Berenberg has upgraded its ra...
5,In a research note published by Sebastian Bray...
6,"Der renommierte Volkswirt Holger Schmieding, d..."
7,In relation to Credit Suisse's emergency takeo...
8,The article features an exclusive interview wi...
9,The article discusses the recommendation made ...


In [12]:
# change datatypes of input data
df_train['news_content']=df_train['news_content'].astype(str)

df_val['news_content']=df_val['news_content'].astype(str)
df_train['news_content'].iloc[0]

train_data=df_train
val_data=df_val

In [15]:
torch.cuda.empty_cache()
gc.collect()

0

In [16]:
import sys
sys.path.append("..")
# Add the parent directory to the Python path
from reghub_pack.models import BERT_RegHub_Similarity

Using pad_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
model = BERT_RegHub_Similarity()
model.load_mlm()
model.model_training(train_data=train_data, val_data=val_data)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model t

Epoch 2/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.98s/item, loss=0.995]


Epoch 3/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:24<00:00, 12.02s/item, loss=0.948]


Epoch 4/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.93s/item, loss=0.904]


Epoch 5/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.98s/item, loss=0.863]


Epoch 6/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:24<00:00, 12.01s/item, loss=0.823]


Epoch 7/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.95s/item, loss=0.786]


Epoch 8/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.96s/item, loss=0.751]


Epoch 9/10: 100%|██████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.99s/item, loss=0.717]


Epoch 10/10: 100%|█████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.98s/item, loss=0.685]


In [ ]:
model.delete_MLM()
model.weight_transfer()
model.save_model(save_aws=False)